In [31]:
import csv
import os
import itertools as itt
import classes as vp
import plot
import mathHelper as mh

In [2]:
def loadSessions(file = 'sessions.csv', csv_data_dir = './csv_data'):
    #load session data form sessions.csv
    sessions = [] #list of all the sessions
    with open(file) as sessions_csv:
        sessionno = 0
        csv_reader = csv.reader(sessions_csv, delimiter=',')
        for l in csv_reader:
            sessions.append(vp.Session(sessionno, l[1], l[2], csv_data_dir + '/' + l[3]))
            sessionno += 1
    return sessions
    

In [3]:
#todo: need output
def loadFactions(file = 'factions.csv'):
    with open(file) as factions_csv:
    lineno = 0
    csv_reader = csv.reader(factions_csv, delimiter=',')
    for line in csv_reader:
        if lineno != 0:
            factions.append(line[1])
            memno.append(line[2])
        lineno += 1

IndentationError: expected an indented block (<ipython-input-3-4df89e1468f9>, line 4)

In [4]:
def parseMems(r, session):
    #members = dict()
    posIdMap = []
    line = next(r)
    start = 0
    end = 0
    for i, item in enumerate(line):
        if item == 'CouncillorId':
            start = i+1
            break
    
    line = next(r)
    
    #enumerate ID
    for i, item in enumerate(line[start:]):
        if item == '':
            end = i + start
            break
            
        posIdMap.append(item)
        if not item in members:
            members[item] = vp.Member(item)
        (members[item].ismem).append(session.idno)

    ''' 
    #enumerate bioid
    line = next(r)    
    for i,item in enumerate(line[start:end]):
        members[posIdMap[i]].bioid = item
    '''
    
    #enumerate name
    line = next(r)
    for i,item in enumerate(line[start:end]):
        #print(i), print(item)
        members[posIdMap[i]].name = item
        
    #skip line (this line only shows members are part of national council)
    next(r)
    #enumerate faction membership
    line = next(r)
    for i,item in enumerate(line[start:end]):
        members[posIdMap[i]].faction = item 
        
    #enumerate canton
    line = next(r)
    for i,item in enumerate(line[start:end]):
        members[posIdMap[i]].canton = item
        
    #enumerate birthdata
    line = next(r)
    for i,item in enumerate(line[start:end]):
        members[posIdMap[i]].birthdate = item

    

In [5]:
#parses all the sessions for the votes
def parseVotes(r, s):
    l = next(r)
    voters = []
    #votes = []
    voteno = 0
    start = 0
    while l[0] != 'VoteDate':
        l = next(r)
        start += 1
    
    #voters contains all the voters in a session
    endVotes = 0
    for i, item in enumerate(l[12:]):
        if item != 'Decision':
            voters.append(item)
        else:
            endVotes = i + 12
            break
    #print(len(voters))
    
    g = set()
    l = next(r)
    
    
    #need to handle:
    vote_none = '' #member was not yet or is no longer a national coucil member
    vote_yes = 'Ja'
    vote_no = 'Nein'
    vote_novote = ['Entschuldigt', 'Hat nicht teilgenommen']
    vote_abstain = 'Enthaltung'
    vote_pres = 'Der Präsident stimmt nicht'
    decision_set = {'yes', 'no'}
    
    
    while True:
        try:
            #store vote meta data
            v = vp.Vote(voteno)
            v.date = l[0]
            v.affairId = l[4]
            v.title = l[5]
            v.session = s
            v.sessionId = s.idno
            #store how each member voted by adding the members bioid to the yes/no/abstain/noset set
            for i, item in enumerate(l[12:endVotes]):
                if item == vote_none:
                    pass
                elif item == vote_yes:
                    v.yes.add(voters[i])
                elif item == vote_no:
                    v.no.add(voters[i])
                elif item == vote_abstain:
                    v.abstain.add(voters[i])
                elif item in vote_novote:
                    v.novote.add(voters[i])
                elif item == vote_pres:
                    pass
                else:
                    print('err vote: ' + item)
            
            #store decision data as stored in file and compare with calculated decision value
            v.decision = l[endVotes].lower()
            
            if v.decision != v.getDecision() or (not v.decision in decision_set):
                v.decision = 'err: \"' + v.decision + '\"' 
            votes.append(v)
            voteno += 1
            l = next(r)
        except StopIteration:
            break

In [6]:
def parseFiles(sessions, toParse = {}): 
    #parse all the files
    #for testing: "filesToParse" can limit the parsed files (everything if empty)
    for s in sessions:
        if s.idno in toParse or len(toParse) == 0:
            with open(s.path) as csv_file:
                csv_reader = csv.reader(csv_file, delimiter=',')
                print('-', end='')
                #print(str(key) + '. reading ... ' + csv_file.name)
                #parseFile(csv_reader)
                members = parseMems(csv_reader, s)
                parseVotes(csv_reader, s)
                csv_file.close

In [85]:
members = dict()
votes = []
factions = dict()
#factions = []
memno = []

In [86]:
sessions = loadSessions()
parseFiles(sessions, toParse={})




---------------------------------------

In [ ]:
for v in votes:
    print(v)

In [ ]:
#count yes/no/draw
y = 0
n = 0
d = 0
for v in votes:
    if v.decision == 'yes':
        y += 1
    elif v.decision == 'no':
        n += 1
    else:
        d += 1
        print(v)
print(y)
print(n)
'''
draw is no?
change .pivotal() for draw rules
president?
2 people missing?
'''
plot.factionBar(['y', 'n', 'd'], [y,n,d])

In [ ]:
#count total members voted
out = []
for v in votes:
    s = len(v.yes) - len(v.no)
    out.append(s)
plot.hist(out)

In [ ]:
#counts the members of each faction per session
factions = dict()
for mem in members:
    f = members[mem].faction
    for s in members[mem].ismem:
        if s in factions:
            if f in factions[s]:
                factions[s][f] += 1
            else:
                factions[s][f] = 1
        else:
            factions[s] = dict()
            factions[s][f] = 1
  


In [87]:
#list of members for each faction
factions = dict()
for mem in members:
    f = members[mem].faction
    if f in factions:
        factions[f].append(members[mem].bioid)
    else:
        factions[f] = [members[mem].bioid]
    

In [77]:
#dict with size of each faction
factions_size = dict()
#factions_size['total'] = 0
for f in factions:
    factions_size[f] = len(factions[f])
    #factions_size['total'] += len(factions[f])

In [83]:
votingPower(factions_size)

{'V': 0.2901960784313726,
 'S': 0.21176470588235294,
 'CE': 0.1411764705882353,
 'RL': 0.10980392156862745,
 'G': 0.0784313725490196,
 'BD': 0.0392156862745098,
 'GL': 0.03137254901960784,
 '-': 0.01568627450980392}

In [81]:
factions_size

{'V': 56, 'S': 47, 'CE': 35, 'RL': 30, 'G': 15, 'BD': 9, 'GL': 8, '-': 2}

In [68]:
list(itt.combinations(factions.keys(), 3))[0]

('V', 'S', 'CE')

In [ ]:
for mem in factions['S']:
    print( mem in votes[2].yes)

In [12]:
'3867' in factions['V']

True

In [88]:
#count how often a party in pivotal
pivotal = dict()
for f in factions:
    count = 0
    for v in votes:
        if v.isPivotal(factions[f]):
            count += 1
    pivotal[f] = count / len(votes)

In [89]:
pivotal

{'V': 0.4825633051976899,
 'S': 0.3553976010661928,
 'C': 0.24478009773434029,
 'CE': 0.023767214571301643,
 'GL': 0.0700799644602399,
 'RL': 0.3175255442025766,
 'G': 0.08907152376721457,
 'BD': 0.05908485117725455,
 '-': 0.0006663705019991116}

In [25]:
factions_size

{'total': 202,
 'V': 56,
 'S': 47,
 'CE': 35,
 'RL': 30,
 'G': 15,
 'BD': 9,
 'GL': 8,
 '-': 2}

In [ ]:
def eqDictSet(d):
    out = set()
    

In [ ]:
factions

In [ ]:
def extendSet(s, x):
    for i in x:
        if i != '' and i != 'CouncillorName':
            s.add(i)

In [ ]:
def extendDict(d, l):
    for i in l:
        if i == '' or i == 'Fraktion':
            continue
        if i in d:
            d[i] +=1
        else:
            d[i] = 1

In [ ]:
def parseFile(file):
    lineno = 0
    for line in file:
        votes.add(line[20])
        if lineno == 2:
            extendSet(members, line)
        if lineno == 4:
            extendDict(faction, line)
        if lineno == 8:
            print(line)
        lineno += 1

In [ ]:
members['2670'].name

In [ ]:
len(set())

In [ ]:
print(members['3008'].ismem)

In [ ]:
mh.dictPrint(members)

In [ ]:
a1 = 1
a2 = 2
max({a1,a2})

In [ ]:
max({1,2})/sum({1,2})